In [56]:
import numpy as np
import pandas as pd
import dtale
import pandas_profiling as pp
from IPython.display import Javascript
from plotly.offline import iplot, init_notebook_mode

In [2]:
# load the dataset
accident_victims_2019_df = pd.read_parquet("../../data/silver/accident_victims_2019_df.parquet")

In [3]:
accident_victims_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85607 entries, 0 to 85606
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DT_DAY                     85607 non-null  datetime64[ns]
 1   DT_HOUR                    85607 non-null  Int64         
 2   CD_DAY_OF_WEEK             85607 non-null  Int64         
 3   TX_DAY_OF_WEEK_DESCR_FR    85607 non-null  string        
 4   TX_DAY_OF_WEEK_DESCR_NL    85607 non-null  string        
 5   MS_VICT                    85607 non-null  Int64         
 6   MS_VIC_OK                  85607 non-null  Int64         
 7   MS_SLY_INJ                 85607 non-null  Int64         
 8   MS_SERLY_INJ               85607 non-null  Int64         
 9   MS_DEAD_30_DAYS            85607 non-null  Int64         
 10  CD_BUILD_UP_AREA           85607 non-null  Int64         
 11  TX_BUILD_UP_AREA_DESCR_NL  85607 non-null  string        
 12  TX_B

In [4]:
# remove any duplicate values
accident_victims_2019_df = accident_victims_2019_df.drop_duplicates().reset_index(drop=True)

# keep only the victim type 'bicycle' users

In [5]:
# remove all other records, keep only the victim type 'bicycle' type 
accident_victims_2019_df = accident_victims_2019_df[(accident_victims_2019_df['TX_VICT_TYPE_DESCR_NL'] == 'Fietser')]

In [6]:
# select only a few important columns (date, vilage, ...)
accident_per_day_2019_df = accident_victims_2019_df[["DT_DAY", "TX_BUILD_UP_AREA_DESCR_NL", "CD_ROAD_USER_TYPE", "TX_ROAD_TYPE_DESCR_NL", "TX_LIGHT_COND_DESCR_NL", "TX_AGE_CLS_DESCR_NL", "TX_MUNTY_DESCR_NL"]]

In [7]:
accident_per_day_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11368 entries, 6 to 85501
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DT_DAY                     11368 non-null  datetime64[ns]
 1   TX_BUILD_UP_AREA_DESCR_NL  11368 non-null  string        
 2   CD_ROAD_USER_TYPE          11368 non-null  Int64         
 3   TX_ROAD_TYPE_DESCR_NL      11368 non-null  string        
 4   TX_LIGHT_COND_DESCR_NL     11368 non-null  string        
 5   TX_AGE_CLS_DESCR_NL        11368 non-null  string        
 6   TX_MUNTY_DESCR_NL          11368 non-null  string        
dtypes: Int64(1), datetime64[ns](1), string(5)
memory usage: 721.6 KB


In [8]:
accident_per_day_2019_df

,DT_DAY,TX_BUILD_UP_AREA_DESCR_NL,CD_ROAD_USER_TYPE,TX_ROAD_TYPE_DESCR_NL,TX_LIGHT_COND_DESCR_NL,TX_AGE_CLS_DESCR_NL,TX_MUNTY_DESCR_NL
6,2019-12-11,Buiten bebouwde kom,25,Gemeenteweg,"Nacht, ontstoken openbare verlichting",35 tot 39 jaar,Aartselaar
7,2019-11-26,Buiten bebouwde kom,18,Gewestweg,Bij klaarlichte dag,55 tot 59 jaar,Aartselaar
13,2019-11-14,Binnen bebouwde kom,18,Gemeenteweg,"Nacht, ontstoken openbare verlichting",35 tot 39 jaar,Aartselaar
16,2019-11-07,Binnen bebouwde kom,18,Gemeenteweg,Dageraad - schemering,55 tot 59 jaar,Aartselaar
17,2019-10-18,Buiten bebouwde kom,25,Gemeenteweg,Bij klaarlichte dag,45 tot 49 jaar,Aartselaar
...,...,...,...,...,...,...,...
85329,2019-07-03,Binnen bebouwde kom,18,Gewestweg,Bij klaarlichte dag,25 tot 29 jaar,Florennes
85373,2019-12-30,Binnen bebouwde kom,18,Gemeenteweg,Bij klaarlichte dag,20 tot 24 jaar,Florennes
85375,2019-11-21,Binnen bebouwde kom,18,Gemeenteweg,Bij klaarlichte dag,10 tot 14 jaar,Florennes
85469,2019-09-04,Binnen bebouwde kom,18,Gemeenteweg,Bij klaarlichte dag,45 tot 49 jaar,Walcourt


## count is the right function to use
but, we need the number of incidents that day

In [9]:
# sort date first, then sort on vilagename, then group these two and count() the number of incidents that occured
# then create a new dataframe and sort the results with the most accidents on a given day first 
groupby_accident_per_day_2019_df = accident_per_day_2019_df.groupby(['DT_DAY', 'TX_MUNTY_DESCR_NL'])['TX_ROAD_TYPE_DESCR_NL'].count().to_frame(name='nr_of_accidents').reset_index()
groupby_accident_per_day_2019_df = groupby_accident_per_day_2019_df.sort_values(by=['nr_of_accidents', 'DT_DAY', 'TX_MUNTY_DESCR_NL'], ascending=[False, True, False])
groupby_accident_per_day_2019_df

,DT_DAY,TX_MUNTY_DESCR_NL,nr_of_accidents
5364,2019-08-26,Antwerpen,13
5487,2019-08-30,Antwerpen,13
6766,2019-10-16,Antwerpen,13
3461,2019-06-17,Antwerpen,12
7919,2019-12-05,Antwerpen,12
...,...,...,...
8338,2019-12-31,Schoten,1
8337,2019-12-31,Roeselare,1
8336,2019-12-31,Middelkerke,1
8335,2019-12-31,Heusden-Zolder,1


In [10]:
# this was to check and verify the correctness of the results - ALL GOOD
groupby_accident_per_day_2019_df[groupby_accident_per_day_2019_df["TX_MUNTY_DESCR_NL"] == 'Antwerpen'].sort_values(by=['DT_DAY'], ascending=[True])

,DT_DAY,TX_MUNTY_DESCR_NL,nr_of_accidents
10,2019-01-02,Antwerpen,1
39,2019-01-04,Antwerpen,3
46,2019-01-05,Antwerpen,2
61,2019-01-07,Antwerpen,4
81,2019-01-08,Antwerpen,3
...,...,...,...
8268,2019-12-24,Antwerpen,1
8282,2019-12-26,Antwerpen,2
8291,2019-12-27,Antwerpen,1
8308,2019-12-29,Antwerpen,2


# so now, we calculate the accidents/cyclists RATIO

In [11]:
# load population values
village_populations_wiki_df = pd.read_parquet("../../data/silver/village_populations_wiki_df.parquet")

In [12]:
# village_populations_wiki_df.columns = ["Village", "Population", "km2_size"]

In [13]:
village_populations_wiki_df

,Village,Population,km2_size
0,Antwerpen,446525,"204,51"
1,Gent,224180,"156,18"
2,Charleroi,200827,"102,08"
3,Luik,185639,"69,39"
4,Brussel,133859,"32,61"
...,...,...,...
576,Martelange,1428,"29,67"
577,Herbeumont,1434,"58,81"
578,Daverdisse,1360,"56,40"
579,Mesen,964,"3,58"


In [14]:
village_populations_wiki_df['km2_size'] = village_populations_wiki_df['km2_size'].str[:-3:].astype(int)

In [15]:
village_populations_wiki_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581 entries, 0 to 580
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Village     581 non-null    string
 1   Population  581 non-null    Int64 
 2   km2_size    581 non-null    int64 
dtypes: Int64(1), int64(1), string(1)
memory usage: 14.3 KB


# now we calculate the ratio

In [16]:
groupby_accident_per_day_2019_df = groupby_accident_per_day_2019_df.replace({'TX_MUNTY_DESCR_NL':{
    'Sint-Niklaas (Sint-Niklaas)': 'Sint-Niklaas',
    'Halle (Halle-Vilvoorde)': 'Halle',
    'Beveren (Sint-Niklaas)': 'Beveren',
    'Tielt (Tielt)': 'Tielt',
    'Vorst (Brussel-Hoofdstad)': 'Vorst',
    'Kapellen (Antwerpen)': 'Kapellen',
    'Hamme (Dendermonde)': 'Hamme',
    'Nieuwerkerken (Hasselt)': 'Nieuwerkerken',
    'Machelen (Halle-Vilvoorde)': 'Machelen',
    'Hove (Antwerpen)': 'Hove',
    'Celles (Doornik)': 'Celles',
    'Perwijs (Nijvel)': 'Perwijs',
    'Saint-Nicolas (Luik)': 'Saint-Nicolas',
    'Aalst (Aalst)': 'Aalst',
    'Moerbeke (Gent)': 'Moerbeke'}})

In [17]:
village_populations_wiki_df = village_populations_wiki_df.replace({'Village':{
    'Pelt*': 'Pelt',
    'Deinze*': 'Deinze',
    'Kruisem*': 'Kruisem',
    'Aalter*': 'Aalter',
    'Belœil': 'Beloeil',
    'Lievegem*': 'Lievegem',
    "Mont-de-l'Enclus": 'Mont-de-l’Enclus',
    'Puurs-Sint-Amands*': 'Puurs-Sint-Amands',
    'Court-Saint-Étienne': 'Court-Saint-Etienne',
    'Éghezée': 'Eghezée',
    'Oudsbergen*': 'Oudsbergen',
    'Estaimpuis*': 'Estaimpuis',
    "Fontaine-l'Evêque": 'Fontaine-l’Evêque',
    'Gembloers': 'Gembloux',
    '’s Gravenbrakel*': '’s Gravenbrakel',
    'Erezée*': 'Erezée',
    'Ecaussines': 'Ecaussinnes',
    'Blegny*': 'Blegny',
    'Plombières*': 'Plombières'}})

In [18]:
# merge join on village
merged_accidents_df = groupby_accident_per_day_2019_df.merge(village_populations_wiki_df, left_on='TX_MUNTY_DESCR_NL', right_on='Village', how='left')

In [19]:
# remove Nan records
merged_accidents_df = merged_accidents_df[~(merged_accidents_df['Population'].isna())]

# calculate the ratio itself

### (nr of incidents / population ) * city size in km

In [20]:
merged_accidents_df.loc[:, 'accident_village_ratio'] = merged_accidents_df['nr_of_accidents'] / merged_accidents_df['Population']

In [21]:
merged_accidents_df.loc[:, 'accident_village_size_ratio'] = merged_accidents_df['accident_village_ratio'] * merged_accidents_df['km2_size']

In [22]:
merged_accidents_df

,DT_DAY,TX_MUNTY_DESCR_NL,nr_of_accidents,Village,Population,km2_size,accident_village_ratio,accident_village_size_ratio
0,2019-08-26,Antwerpen,13,Antwerpen,446525,204.0,0.000029,0.005939
1,2019-08-30,Antwerpen,13,Antwerpen,446525,204.0,0.000029,0.005939
2,2019-10-16,Antwerpen,13,Antwerpen,446525,204.0,0.000029,0.005939
3,2019-06-17,Antwerpen,12,Antwerpen,446525,204.0,0.000027,0.005482
4,2019-12-05,Antwerpen,12,Antwerpen,446525,204.0,0.000027,0.005482
...,...,...,...,...,...,...,...,...
8336,2019-12-31,Schoten,1,Schoten,32733,29.0,0.000031,0.000886
8337,2019-12-31,Roeselare,1,Roeselare,54199,59.0,0.000018,0.001089
8338,2019-12-31,Middelkerke,1,Middelkerke,16583,75.0,0.00006,0.004523
8339,2019-12-31,Heusden-Zolder,1,Heusden-Zolder,30105,53.0,0.000033,0.001761


In [23]:
accident_village_ratio_median_cutoff = merged_accidents_df['accident_village_ratio'].median()
accident_village_ratio_median_cutoff

3.9201850327335454e-05

In [24]:
accident_village_size_ratio_median_cutoff = merged_accidents_df['accident_village_size_ratio'].median()
accident_village_size_ratio_median_cutoff

0.0018274452718212866

In [25]:
# chart_data = pd.concat([
#     merged_accidents_df['Village'],
#     merged_accidents_df['accident_village_size_ratio'],
# ], axis=1)
# chart_data = chart_data.sort_values(['Village'])
# chart_data = chart_data.rename(columns={'Village': 'x'})
# chart_data = chart_data.dropna()

# import plotly.graph_objs as go

# charts = []
# charts.append(go.Bar(
#     x=chart_data['x'],
#     y=chart_data['accident_village_size_ratio']
# ))
# figure = go.Figure(data=charts, layout=go.Layout({
#     'barmode': 'stack',
#     'legend': {'orientation': 'h'},
#     'title': {'text': 'accident_village_size_ratio by Village'},
#     'xaxis': {'title': {'text': 'Village'}},
#     'yaxis': {'title': {'text': 'accident_village_size_ratio'}, 'type': 'linear'}
# }))

In [26]:
# from plotly.offline import iplot, init_notebook_mode

# init_notebook_mode(connected=True)
# for chart in charts:
#     chart.pop('id', None) # for some reason iplot does not like 'id'
# iplot(figure)

In [41]:
# # generate a data report to get a feel of the data (EDA)
# d = dtale.show(merged_accidents_df, host="20.223.36.46", port="40000", ignore_duplicate=True, drop_index=True, reaper_on=False)
# # show all running instances
# d.main_url()

http://20.223.36.46:40000/dtale/main/1


In [27]:
result = merged_accidents_df.groupby(['Village'])['accident_village_size_ratio'].sum().to_frame(name='ratio').reset_index()
result = result.sort_values(by=['ratio', 'Village'], ascending=[False, True])
result.head(10)

,Village,ratio
18,Antwerpen,0.559198
165,Gent,0.477366
77,Brugge,0.37033
51,Beveren,0.323499
306,Maldegem,0.298602
224,Ieper,0.285421
418,Sint-Niklaas,0.280759
113,Diksmuide,0.202616
190,Hasselt,0.200829
278,Leuven,0.195813


In [71]:
# generate a data report to get a feel of the data (EDA)
d = dtale.show(result, host="20.223.36.46", port="40000", ignore_duplicate=True, drop_index=True, reaper_on=False)
# show all running instances
d.main_url()

http://20.223.36.46:40000/dtale/main/2
Exception occurred while processing request: object arrays are not supported
Traceback (most recent call last):
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/views.py", line 121, in _handle_exceptions
    return func(*args, **kwargs)
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/views.py", line 2680, in get_column_analysis
    return jsonify(**analysis.build())
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/column_analysis.py", line 137, in build
    return_data, code = self.analysis.build(self)
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/column_analysis.py", line 184, in build
    kde, kde_code = build_kde(
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/column_analysis.py", line 67, in build_kde
    kde = sts.gaussian_kde(s)
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/scipy/stats/_kde.py", l

2022-03-27 12:19:41,982 - ERROR    - Exception occurred while processing request: object arrays are not supported
Traceback (most recent call last):
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/views.py", line 121, in _handle_exceptions
    return func(*args, **kwargs)
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/views.py", line 2680, in get_column_analysis
    return jsonify(**analysis.build())
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/column_analysis.py", line 137, in build
    return_data, code = self.analysis.build(self)
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/column_analysis.py", line 184, in build
    kde, kde_code = build_kde(
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/column_analysis.py", line 67, in build_kde
    kde = sts.gaussian_kde(s)
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/scipy/stats/_kde.py", lin

Exception occurred while processing request: object arrays are not supported
Traceback (most recent call last):
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/views.py", line 121, in _handle_exceptions
    return func(*args, **kwargs)
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/views.py", line 2680, in get_column_analysis
    return jsonify(**analysis.build())
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/column_analysis.py", line 137, in build
    return_data, code = self.analysis.build(self)
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/column_analysis.py", line 184, in build
    kde, kde_code = build_kde(
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/column_analysis.py", line 67, in build_kde
    kde = sts.gaussian_kde(s)
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/scipy/stats/_kde.py", line 207, in __init__
    self.set_bandw

2022-03-27 12:20:05,441 - ERROR    - Exception occurred while processing request: object arrays are not supported
Traceback (most recent call last):
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/views.py", line 121, in _handle_exceptions
    return func(*args, **kwargs)
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/views.py", line 2680, in get_column_analysis
    return jsonify(**analysis.build())
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/column_analysis.py", line 137, in build
    return_data, code = self.analysis.build(self)
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/column_analysis.py", line 184, in build
    kde, kde_code = build_kde(
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/dtale/column_analysis.py", line 67, in build_kde
    kde = sts.gaussian_kde(s)
  File "/anaconda/envs/py38_datareport/lib/python3.8/site-packages/scipy/stats/_kde.py", lin

In [53]:
s = result['ratio']
q1 = s.quantile(0.25)
q3 = s.quantile(0.75)
iqr = q3 - q1
iqr_lower = q1 - 1.5 * iqr
iqr_upper = q3 + 1.5 * iqr
outliers = dict(s[(s < iqr_lower) | (s > iqr_upper)])

In [54]:
result.iloc[list(outliers.keys())]

,Village,ratio
245,Knokke-Heist,0.155424
59,Bocholt,0.044671
379,Ravels,0.071761
24,Assenede,0.096104
313,Meise,0.020385
80,Buggenhout,0.030905
194,Hemiksem,0.008196
432,Stekene,0.058553
477,Wellen,0.038662
330,Moorslede,0.022827


In [64]:
chart_data = pd.concat([
    result.iloc[list(outliers.keys())]['Village'],
    result.iloc[list(outliers.keys())]['ratio'],
], axis=1)
chart_data = chart_data.sort_values(['Village'])
chart_data = chart_data.rename(columns={'Village': 'x'})
chart_data = chart_data.dropna()

import plotly.graph_objs as go

charts = []
charts.append(go.Bar(
    x=chart_data['x'],
    y=chart_data['ratio']
))
figure = go.Figure(data=charts, layout=go.Layout({
    'barmode': 'stack',
    'legend': {'orientation': 'h'},
    'title': {'text': 'Outliers (highest & lowest)'},
    'xaxis': {'title': {'text': 'Cities'}},
    'yaxis': {'title': {'text': 'ratio of cylists'}, 'type': 'linear'}
}))

In [70]:
init_notebook_mode(connected=True)
for chart in charts:
    chart.pop('id', None) # for some reason iplot does not like 'id'
iplot(figure)

In [67]:
figure.write_image("../../data/plot/accident_ratio_bicycles.png")

In [72]:
reportFile = "../../data/report/accident_ratio_bicycles_report.html"

In [73]:
# quick report on 100% records (no correlation matrix stuff)
pp.ProfileReport(df=result.sample(frac=1),
                 minimal=True,
                 progress_bar=False,
                 correlations={"cramers": {"calculate": False}},
                 title="Report Title",
                 html={"style": {"full_width": True}}).to_file(reportFile)

In [74]:
# open the report (*.html)
display(Javascript('window.open("{url}");'.format(url=reportFile)))

<IPython.core.display.Javascript object>